In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
end_use = "naics?get=ALL_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_exports"] = df["ALL_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_exports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_exports", ascending = False).CTY_CODE)[0:21]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [4]:
country_list[0] = ""

In [5]:
country_list

['',
 '1220',
 '2010',
 '5700',
 '5880',
 '4120',
 '4280',
 '5800',
 '4210',
 '3510',
 '5820',
 '4279',
 '4231',
 '5590',
 '5830',
 '5330',
 '6021',
 '4419',
 '5200',
 '4759',
 '5170']

In [8]:
date = "2020-11"

end_use = "hs?get=CTY_NAME,ALL_VAL_MO,E_COMMODITY,E_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

surl  = surl + my_key + "&time=" + date + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\exports\\" + xxx + "-" + date + "-data.parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\exports\\" + "TOTAL" + "-" + date + "-data.parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)


<Response [200]>
1220
<Response [200]>
2010
<Response [200]>
5700
<Response [200]>
5880
<Response [200]>
4120
<Response [200]>
4280
<Response [200]>
5800
<Response [200]>
4210
<Response [200]>
3510
<Response [200]>
5820
<Response [200]>
4279
<Response [200]>
4231
<Response [200]>
5590
<Response [200]>
5830
<Response [200]>
5330
<Response [200]>
6021
<Response [200]>
4419
<Response [200]>
5200
<Response [200]>
4759
<Response [200]>
5170
<Response [200]>


In [12]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])

for xxx in country_list:
    
    file = ".\\data\\exports\\" + xxx + "data.parquet"

    foo = pq.read_table(file).to_pandas()
    
    df = df.append(foo)
    
    recentfile = ".\\data\\exports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    foo = pq.read_table(recentfile).to_pandas()
    
    df = df.append(foo)
    
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["exports"] = df["ALL_VAL_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"exports": "sum"})
                   
total.reset_index(inplace = True)
                   
total["E_COMMODITY"] = "00"
total["E_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)                   

df.drop(["ALL_VAL_MO", "COMM_LVL", "CTY_CODE"], axis = 1, inplace = True)

df["E_COMMODITY_SDESC"] = "HS CODE " + df["E_COMMODITY"] + ", " + df["E_COMMODITY_SDESC"]

df["E_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","E_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top20-HS2-exports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [14]:
df.loc["JAPAN"]

E_COMMODITY  \
E_COMMODITY_SDESC                                  time                     
HS CODE 03, FISH, CRUSTACEANS & AQUATIC INVERTE... 2013-01-01          03   
                                                   2013-02-01          03   
                                                   2013-03-01          03   
                                                   2013-04-01          03   
                                                   2013-05-01          03   
...                                                                   ...   
ALL PRODUCTS                                       2020-07-01          00   
                                                   2020-08-01          00   
                                                   2020-09-01          00   
                                                   2020-10-01          00   
                                                   2020-11-01          00   

                                                                    exports  
E_COMMODITY_SDESC                                  time                      
HS CODE 03, FISH, CRUSTACEANS & AQUATIC INVERTE... 2013-01-01  2.079056e+07  
                                                   2013-02-01  3.952987e+07  
                                                   2013-03-01  5.093972e+07  
                                                   2013-04-01  6.001953e+07  
                                                   2013-05-01  6.068606e+07  
...                                                                     ...  
ALL PRODUCTS                                       2020-07-01  4.846875e+09  
                                                   2020-08-01  5.205089e+09  
                                                   2020-09-01  4.841034e+09  
                                                   2020-10-01  5.401048e+09  
                                                   2020-11-01  5.035796e+09  

[9300 rows x 2 columns]

In [3]:
df = pq.read_table(".\\data"+ "\\top20-HS2-exports.parquet").to_pandas()

foo = df.loc[(["MEXICO", "UNITED KINGDOM"], "ALL PRODUCTS"),:]

In [4]:
country_list = ["MEXICO", "UNITED KINGDOM"]

In [6]:
[foo.loc[(name, "ALL PRODUCTS"),"exports"].values for name in country_list]

[array([1.79024537e+10, 1.77032021e+10, 1.80927482e+10, 1.97945826e+10,
        1.92261881e+10, 1.79863136e+10, 1.95818245e+10, 1.91911208e+10,
        1.80230202e+10, 2.10653287e+10, 1.92904216e+10, 1.80971638e+10,
        1.91674146e+10, 1.84678328e+10, 2.00152835e+10, 1.99003060e+10,
        2.10590632e+10, 2.01518677e+10, 2.08328242e+10, 2.04533683e+10,
        2.00652660e+10, 2.23490541e+10, 1.97067058e+10, 1.88381762e+10,
        1.91408660e+10, 1.81598700e+10, 1.98897094e+10, 2.00468000e+10,
        1.96935296e+10, 2.05213255e+10, 2.11555908e+10, 1.95940488e+10,
        1.95492483e+10, 2.12373645e+10, 1.89084632e+10, 1.85633211e+10,
        1.80454611e+10, 1.81309069e+10, 1.92945440e+10, 1.93621627e+10,
        1.89914746e+10, 1.94562513e+10, 1.83474306e+10, 2.00339113e+10,
        1.97199427e+10, 2.02092127e+10, 1.95669731e+10, 1.90705627e+10,
        1.96537723e+10, 1.82308206e+10, 2.10386098e+10, 1.89354755e+10,
        1.99096853e+10, 2.14025658e+10, 1.98489292e+10, 2.088696

In [13]:
def growth_trade(foo):
    # what this function does is take a dataframe and create a relative 
    foo["growth"] = 100*((foo["exports"]/foo["exports"].shift(12)) - 1)    
    
    return foo

In [18]:
foo.groupby(["CTY_NAME"]).apply(growth_trade)

E_COMMODITY       exports  \
CTY_NAME       E_COMMODITY_SDESC time                                   
MEXICO         ALL PRODUCTS      2013-01-01          00  1.790245e+10   
                                 2013-02-01          00  1.770320e+10   
                                 2013-03-01          00  1.809275e+10   
                                 2013-04-01          00  1.979458e+10   
                                 2013-05-01          00  1.922619e+10   
...                                                 ...           ...   
UNITED KINGDOM ALL PRODUCTS      2020-07-01          00  4.092462e+09   
                                 2020-08-01          00  4.743759e+09   
                                 2020-09-01          00  5.025366e+09   
                                 2020-10-01          00  5.445534e+09   
                                 2020-11-01          00  5.366735e+09   

                                                growth  
CTY_NAME       E_COMMODITY_SDESC time                   
MEXICO         ALL PRODUCTS      2013-01-01        NaN  
                                 2013-02-01        NaN  
                                 2013-03-01        NaN  
                                 2013-04-01        NaN  
                                 2013-05-01        NaN  
...                                                ...  
UNITED KINGDOM ALL PRODUCTS      2020-07-01 -22.783541  
                                 2020-08-01 -19.201016  
                                 2020-09-01 -14.223209  
                                 2020-10-01 -10.136329  
                                 2020-11-01 -14.997820  

[190 rows x 3 columns]

In [16]:
grp = foo.groupby(["CTY_NAME","time"])

In [17]:
grp.get_group("MEXICO")

ValueError: must supply a tuple to get_group with multiple grouping keys